# ground truth for foundpose

generate ground truth pose from COLMAP reconstructed camera poses + manually found world pose

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
from pathlib import Path
from typing import List

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
import transforms3d as t3d
import trimesh
import visu3d as v3d
import yaml

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_model, to_contour
from burybarrel.camera import load_v3dcams

In [ ]:
name = "barrelddt1"
datadir = Path(f"/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/data/input_data/{name}")
resdir = Path(f"/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results/{name}")

infopath = Path("../configs/blender_gt_info.yaml")
with open(infopath, "rt") as f:
    allinfo = yaml.safe_load(f)
info = allinfo[name]
model_path = Path(info["model_path"])
camposes_path = Path(info["camposes_path"])
cams, imgpaths = load_v3dcams(camposes_path, img_parent=None)
scalefactor = info["scalefactor"]
R_gt = t3d.euler.euler2mat(*(np.array(info["R"]) * np.pi / 180))
t_gt = np.array(info["t"]) * (1 / scalefactor)
T_gt = np.eye(4)
T_gt[:3, :3] = R_gt
T_gt[:3, 3] = t_gt
T_gt = v3d.Transform.from_matrix(T_gt)
imgs = np.array([cv2.cvtColor(cv2.imread(imgpath), cv2.COLOR_BGR2RGB) for imgpath in imgpaths])

In [ ]:
mesh = trimesh.load(model_path)
vtxs = np.array(mesh.vertices)
rgb = np.zeros_like(vtxs, dtype=np.uint8)
rgb[:, 0] = 255
vtxs_p3d = v3d.Point3d(p=vtxs, rgb=rgb)

In [ ]:
# scaled 3d
camscaled = cams.replace(world_from_cam=cams.world_from_cam.replace(t=cams.world_from_cam.t * (1 / scalefactor)))
gtoverlaydir = datadir / "gt-overlays"
gtoverlaydir.mkdir(exist_ok=True)
gttmp = gtoverlaydir / "tmp"
gttmp.mkdir(exist_ok=True)
for i, img in enumerate(tqdm(imgs)):
    imgpath = imgpaths[i]
    vtxs_trf = T_gt @ vtxs_p3d
    rgb, _, _ = render_model(camscaled[i], mesh, T_gt, light_intensity=200.0)
    Image.fromarray(rgb).save(gttmp / f"{imgpath.stem}.png")
    overlayimg = to_contour(rgb, color=(255, 0, 0), background=img)
    Image.fromarray(overlayimg).save(gtoverlaydir / f"{imgpath.stem}.png")
    # Image.fromarray(render_v3d(camscaled[i], vtxs_trf, radius=4, background=img)).save(gtoverlaydir / f"{imgpaths[i].stem}.png")

In [ ]:
obj2cams_truth = [cam.world_from_cam.inv @ T_gt for cam in camscaled]
gt_data_list = []
for i, T in enumerate(obj2cams_truth):
    truthdata = {
        "img_path": str(imgpaths[i]),
        "R": T.R.tolist(),
        "t": T.t.tolist(),
    }
    gt_data_list.append(truthdata)
with open(datadir / "gt_obj2cam.json", "wt") as f:
    json.dump(gt_data_list, f)